# Ecosystem Integrity Index score (Dias et al., 2022)

This notebook is for computation of Ecosystem Integrity Index score (EII score) for a set of forested test lands called SEOME plots which are distributed around the globe, each having equal area (5 km2), but are covered with different quality of forest.

In order to run the code, a BII map and a plantations map should be downloaded and added as personal Google Earth Engine (GEE) asset. \
The data for the BII map is available here: https://data.nhm.ac.uk/dataset/global-map-of-the-biodiversity-intactness-index-from-newbold-et-al-2016-science/resource/8531b4dc-bd44-4586-8216-47b3b8d60e85 

The data for the plantations map is available here:
https://www.arcgis.com/home/item.html?id=224e00192f6d408fa5147bbfc13b62dd

In [3]:
import ee
import geemap as gm

# only needed for exporting data to csv
import geopandas
import pandas as pd

from google.cloud import storage

In [4]:
ee.Initialize()

## Import datasets used
First import the datasets used for computing EII.\
\
*NB*: in order to run the code, replace the path in bii and plantations variable with a path to the respective map.
\

In [5]:
# human footprint
hf = ee.ImageCollection('CSP/HM/GlobalHumanModification')

# hansen forest
gfc = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')

# biodoversity intactness index
"""
NB this dataset is not freely available in Google Earth Engine but can be downloaded from this link:
https://data.nhm.ac.uk/dataset/global-map-of-the-biodiversity-intactness-index-from-newbold-et-al-2016-science/resource/8531b4dc-bd44-4586-8216-47b3b8d60e85
In order to run this code, replace the path to your own asset.
"""
bii = ee.Image("add/your/path").rename('BII')

# net primary production from Modis at 500m resolution
npp = ee.ImageCollection("MODIS/006/MOD17A3HGF").\
                  filter(ee.Filter.date('2015-01-01', '2020-01-01')).\
                  select('Npp');


# this is forest total_connectivity(21) :
connectivity_new = ee.Image('users/aduncan/osm_earth/total_connectivity_PRE2021_borealfixed')
# this is forest total_connectivity_original() :
connectivity_orig = ee.Image('users/aduncan/osm_earth/total_connectivity_original_borealfixed')

# add plantations to exclude them 
"""
NB this dataset is not freely available in Google Earth Engine but can be downloaded from this link:
https://www.arcgis.com/home/item.html?id=224e00192f6d408fa5147bbfc13b62dd
In order to run this code, replace the path to your own asset.
"""
plantations = ee.FeatureCollection('add/your/path')


Then import datasets for evaluating EII score results

In [6]:
# import biomes, ecoregions and realms
biogeo = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")

# biomass dataset - only used for testing, not for eii calculation
biomass = ee.Image("LARSE/GEDI/GEDI04_B_002").select('MU')

# digital elevation model - only used for testing, not for eii calculation
# The Global Multi-resolution Terrain Elevation Data 2010 
dem = ee.Image('USGS/GMTED2010').select('be75')

# forest canopy height in 2020 - for testing average canopy of the plot
canopyheight_20 = ee.Image('projects/glad/GLCLU2020/Forest_height_2020')

## Prepare the mask for the intact areas of the ecoregion

The combination of small human footprint areas and hansen forest mask are used to later select global intact areas to find the 90th percentile value for each EBV. Plantations are removed from the intact areas.


In [7]:
# Selecting pixels with high integrity by creating a binary mask. For now it includes all ecosystem (forest grassland etc)
hf_mask = hf.select('gHM').first().lt(0.4);

# create a mask for removing plantations
plantations_ras = plantations.reduceToImage( reducer = ee.Reducer.first(), properties = ['OBJECTID'])

# hansen intact forest
treeCover = gfc.select(['treecover2000'])
hansen_treecover = treeCover.gt(30)
# add plantations as a band
lossImage = gfc.select(['loss']).addBands(plantations_ras.select('first').gt(0))
hansen_noloss_1 = lossImage.mask(hansen_treecover).eq(1); 
# filter out pixels with loss = 1
hansen_noloss_plant = hansen_noloss_1.updateMask(hansen_noloss_1.select('loss').eq(0))
# mask out plantations - this is the final forest mask
hansen_noloss = hansen_noloss_plant.updateMask(hansen_noloss_plant.select('first').eq(0)).select('loss')

## Prepare the bii, npp and lfc layers

For each EBV create two layers with 1000 m resolution - one for high integrity forest where human footprint is less than 0.4 and the other for all areas which are inside forest ecosystem.

First add biodiversity intactness index

In [8]:
# masking bii by the high integrity pixels previously defined. BII layer is already at 1000 m resolution
bii_high_integrity = bii.updateMask(hf_mask)

Then, add net primary production and aggregate it to 1000 m resolution

In [21]:
# Reduce the collection with a using std reducer.
mean_npp = npp.reduce(ee.Reducer.mean());
mean_npp = mean_npp.rename('NPP')

# aggregating NPP to the 1km res
npp = mean_npp.\
    reproject(
      crs = hf.first().projection(),
      scale = 1000
    ).\
    reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 1024
    )

# masking npp by the high integrity pixels previously defined
npp_high_integrity = npp.updateMask(hf_mask)


Add loss of forest connectivity

In [22]:
# changed the variable names to avoid bugs, before all 3 were 'var lci'
lci_0 = connectivity_new.divide(connectivity_orig.unmask(0.001).where(connectivity_orig.eq(0),0.001))
lci_1 = lci_0.where(lci_0.gt(1),1)
lci = lci_1.rename('LFC')

# aggregate to 1km resolution and mask according to Hansen mask
lfc = lci.\
    reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 1024
    ).\
    reproject(
      crs = hf.first().projection(),
      scale = 1000
    )


# mask lfc by the high integrity pixels previously defined
lfc_high_integrity = lfc.updateMask(hf_mask)


Add bands from high integrity bii, lfc and npp values to 30 m hansen intact forest. This creates the final layers from which the 90th percentile values for the 3 components are calculated.

In [23]:
hansen_bii = hansen_noloss.addBands(bii_high_integrity).updateMask(hansen_noloss.select('loss').eq(0)).select('BII')
hansen_npp = hansen_noloss.addBands(npp_high_integrity).updateMask(hansen_noloss.select('loss').eq(0)).select('NPP')
hansen_lfc = hansen_noloss.addBands(lfc_high_integrity).updateMask(hansen_noloss.select('loss').eq(0)).select('LFC')

# Get the global 90th percentile values

This part is to show how the 90th percentile values of bii, npp and lfc from the global low human footprint plantation-free Hansen intact forest were calculated. These values were exported to Google Cloud Storage (GCS) and the next part of the code uses already values retrieved from the GCS.

In [24]:
def percentile_task(image, aoi, asset_id):
    ''' Compute the 90th percentile of an image over the given geometry
    '''
    percentile = image.reduceRegion(
        reducer = ee.Reducer.percentile([90]),
        geometry = aoi,
        crs='EPSG:4326',
        crsTransform=[0.00025, 0, -180, 0, -0.00025, 80],
        maxPixels = 1e15
    )
    f = ee.Feature(aoi).set(percentile)
    fc = ee.FeatureCollection(f)
    
    task = ee.batch.Export.table.toAsset(
        collection=fc,
        description=f'90th percentile',
        assetId=asset_id
    )
    return task

In [25]:
# the geometry covering the entire world
aoi =  ee.Geometry.BBox(-180, -60, 180, 80)

Uncomment task.start() to submit the tasks. May take about 3 hours

In [26]:
task = percentile_task(hansen_bii, aoi, 'add/your/path')
# task.start()

In [27]:
task = percentile_task(hansen_npp, aoi, 'add/your/path')
# task.start()

In [28]:
task = percentile_task(hansen_lfc, aoi, 'add/your/path')
# task.start()

Read back the values. Otherwise, they are expected to be the following and can be hard-coded:\
BII 90th percentile 1.0022686980875006\
NPP 90th percentile 12735.439316547474\
LFC 90th percentile 1

In [29]:
"""
Uncomment if retrieve values from feature saved to GCS
fc_bii = 'enter/path/'
fc_npp = 'enter/path/'
fc_lfc = 'enter/path/'

percentile_bii = ee.FeatureCollection(fc_bii).reduceColumns(ee.Reducer.first(), ['BII']).getInfo()['first']
percentile_npp = ee.FeatureCollection(fc_npp).reduceColumns(ee.Reducer.first(), ['NPP']).getInfo()['first']
percentile_lfc = ee.FeatureCollection(fc_lfc).reduceColumns(ee.Reducer.first(), ['LFC']).getInfo()['first']
"""
percentile_bii = 1.0022686980875006
percentile_npp = 12735.439316547474
percentile_lfc = 1

print('BII 90th percentile', percentile_bii)
print('NPP 90th percentile', percentile_npp)
print('LFC 90th percentile', percentile_lfc)

BII 90th percentile 1.0022686980875006
NPP 90th percentile 12735.439316547474
LFC 90th percentile 1


Once the 90th percentile values have been retrieved, calculate the map of each EBV relative values against the respective 90th percentile

In [30]:
bii = bii.set({'90th_percentile': percentile_bii})
npp = npp.set({'90th_percentile': percentile_npp})
lfc = lfc.set({'90th_percentile': percentile_lfc})

bii_relative = bii.where(bii.gt(percentile_bii), percentile_bii).divide(percentile_bii)
npp_relative = npp.where(npp.gt(percentile_npp), percentile_npp).divide(percentile_npp)
lfc_relative = lfc.where(lfc.gt(percentile_lfc), percentile_lfc).divide(percentile_lfc)

## Show on map

In [33]:
Map = gm.Map(zoom=5)
Map.setOptions('SATELLITE')

Map.addLayer(bii,{min:0, max:1, 'palette': ['white','blue']}, 'bii', False)
Map.addLayer(npp, {min:1000, max:20000, 'palette': ['white','red']}, 'npp', False)
Map.addLayer(lfc, {min:0, max:1, 'palette': ['white','yellow']}, 'lfc', False)

Map.addLayer(hansen_noloss, {'palette': 'green'}, 'Hansen', False)
Map.addLayer(plantations_ras, {'palette': 'red'}, 'plantations mask', False)
Map.addLayer(hansen_bii, {min:0, max:1, 'palette': ['white','blue']}, 'bii intact', False)
Map.addLayer(hansen_npp, {min:1000, max:12700, 'palette': ['white','red']}, 'npp intact', False)
Map.addLayer(hansen_lfc, {min:0, max:1, 'palette': ['white','yellow']}, 'lfc intact', False)

Map.addLayer(bii_relative, {min:0, max:1, 'palette': ['white','blue']}, 'relative bii')
Map.addLayer(npp_relative, {min:1000, max:12700, 'palette': ['white','red']}, 'relative npp')
Map.addLayer(lfc_relative, {min:0, max:1, 'palette': ['white','yellow']}, 'relative lfc')
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Function to stack bii, npp and lfc to one layer with three bands

In [34]:
def appendBands(image, previous):
    return(ee.Image(previous).addBands(image))

def stackCollection(collection):
    first = ee.Image(collection.first()).select([])
    return(ee.Image(collection.iterate(appendBands, first)))


## Calculate EII score for each plot and assess the sensitivity

Function to iterate over features, calculate the EII score for each feature and add it as its property. The function assumes round-shaped plots as it finds the center of the plot and creates new plots of 1, 2, 3 and 4 km2 around the center to test for the sensitivity of EII score to area. The sensitivity to the weights of the EBVs is tested by calculating the EII score using different weights.

In [35]:
def eii_sensitivity_seome(feature): 
    
    # 1. PREPARE DATA ON ECOREGION
    # This is for understanding EII score variabilty within and between ecoregions
    
    # find the biome where the plot is in
    biome_name = ee.String(ee.FeatureCollection(biogeo.filterBounds(feature.geometry())).first().get('BIOME_NAME'))
    # find the realm in which the plot is in
    realm = ee.String(ee.FeatureCollection(biogeo.filterBounds(feature.geometry())).first().get('REALM'))
    # find the ecoregion in which the plot is in
    ecoregion_nr = ee.Number(ee.FeatureCollection(biogeo.filterBounds(feature.geometry())).first().get('ECO_ID'))
    ecoregion_name = ee.String(ee.FeatureCollection(biogeo.filterBounds(feature.geometry())).first().get('ECO_NAME'))
    biome_num = ee.Number(ee.FeatureCollection(biogeo.filterBounds(feature.geometry())).first().get('BIOME_NUM'))
    

    # 2. PREPARE EII IMAGE 
    
    # if any of the mean components is missing, they should for now be 0 - this is to avoid crashing
    mean_bii = ee.Image(ee.Algorithms.If(bii_relative,  bii_relative, ee.Image.constant(0)))
    mean_NPP = ee.Image(ee.Algorithms.If(npp_relative, npp_relative,  ee.Image.constant(0)))
    mean_lfc = ee.Image(ee.Algorithms.If(lfc_relative, lfc_relative,  ee.Image.constant(0)))
    
    # add all 3 images together
    eiiCollection = ee.ImageCollection.fromImages(
        [mean_bii, mean_NPP, mean_lfc]);
    
    # create masks for areas where either of the 3 components is missing 
    noDataMask_bii = (ee.Image(stackCollection(eiiCollection).select('BII')).unmask(-99))
    noDataMask_npp = (ee.Image(stackCollection(eiiCollection).select('NPP')).unmask(-99))
    noDataMask_lfc = (ee.Image(stackCollection(eiiCollection).select('LFC')).unmask(-99))

    
    
    # 3. CALCULATE THE EII SCORE FOR THE ORIGINAL AREA OF THE PLOT (5KM2)
    
    # use function defined above to take the mean of the 3 components
    # then use the 3 masks to mask out any pixels that miss any of the 3 values
    
    eii_average = stackCollection(eiiCollection).reduce('mean').updateMask(noDataMask_bii).\
    updateMask(noDataMask_npp).updateMask(noDataMask_lfc)

    # find the mean of eii of the plot - for analysis
    eii_plot_mean = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = feature.geometry(),
    scale = 1000
  )).get('mean'));
    
    # find the eii sum of the plot - the EII score
    eii_plot_sum = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = feature.geometry(),
    scale = 1000
  )).get('mean'));
    
    

    # 4. ADD THE MEAN BIOMASS, MEAN CANOPY HEIGHT AND ELEVATION OF THE PLOT FOR ANALYSIS
    
    biomass_plot = ee.Number((biomass.updateMask(noDataMask_bii).\
    updateMask(noDataMask_npp).updateMask(noDataMask_lfc).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = feature.geometry(),
    scale = 1000
  )).get('MU'));
    
    # only to test if plots elevation introduces noise
    elevation = dem.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 90
    )
    
    # find the canopy height of the plot
    mean_canopy = ee.Number(canopyheight_20.updateMask(noDataMask_bii).\
    updateMask(noDataMask_npp).updateMask(noDataMask_lfc).reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 30).get('b1')
    )
    
    
    
    
    # 5. SENSITIVITY ANALYSIS TO CHANGE THE WEIGHTS OF THE 3 COMPONENTS
    
    # first if one of the components weighs 50% and the other two 25%
    bii50 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.5)).\
                               add((mean_NPP).multiply(ee.Number(0.25))).\
                               add(mean_lfc.multiply(ee.Number(0.25)))).\
                      # mask out the pixels that are missing either of the 3 values
                      updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII'))
    
    # if npp has 50% weightand the other two 25%
    npp50 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.25)).\
                               add((mean_NPP).multiply(ee.Number(0.5))).\
                               add(mean_lfc.multiply(ee.Number(0.25)))).\
                      updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII')) # get bii as the npp and lfc values were added to the bii value
    
    # if lfc has 50% weight and the other two 25%
    lfc50 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.25)).\
                               add((mean_NPP).multiply(ee.Number(0.25))).\
                               add(mean_lfc.multiply(ee.Number(0.5)))).\
                      updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII'))
    
    # then, if one of the component is 0 and the other 2 is 50%
    bii0 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.0)).\
                              add((mean_NPP).multiply(ee.Number(0.5))).\
                              add(mean_lfc.multiply(ee.Number(0.5)))).\
                     updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII'))
    
    # if npp is 0 and the other 2 is 50%
    npp0 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.5)).\
                              add((mean_NPP).multiply(ee.Number(0.0))).\
                              add(mean_lfc.multiply(ee.Number(0.5)))).\
                     updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII'))
    
    # if lfc is 0 and the other 2 is 50%
    lfc0 = ee.Number(ee.Image(mean_bii.multiply(ee.Number(0.5)).\
                              add((mean_NPP).multiply(ee.Number(0.5))).\
                              add(mean_lfc.multiply(ee.Number(0.0)))).\
                     updateMask(noDataMask_bii).updateMask(noDataMask_npp).updateMask(noDataMask_lfc).\
    reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 1000
    ).get('BII'))
    
    
    
    # 6. SENSITIVITY ANALYSIS TO THE CHANGING AREA OF THE PLOT
    
    #get the center point of the plot, then create buffers around the center
    center = feature.centroid();
    
    # then create 1, 2, 3, 4 km2 buffers around the plot's center
    buf1 = center.buffer(ee.Number(1e6).sqrt().divide(1.7725267552), 1)
    
    #calculate the average eii in the respective buffer
    eii1 = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = buf1.geometry(),
    scale = 1000
  )).get('mean'));
    
    buf2 = center.buffer(ee.Number(2e6).sqrt().divide(1.7725267552), 1)
    
    eii2 = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = buf2.geometry(),
    scale = 1000
  )).get('mean'));
    
    buf3 = center.buffer(ee.Number(3e6).sqrt().divide(1.7725267552), 1)
    
    eii3 = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = buf3.geometry(),
    scale = 1000
  )).get('mean'));
    
    buf4 = center.buffer(ee.Number(4e6).sqrt().divide(1.7725267552), 1)
    
    eii4 = ee.Number((eii_average.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = buf4.geometry(),
    scale = 1000
  )).get('mean'));
    

    return(feature.set({'realm': realm, 'biome': biome_name, 'ecoregion': ecoregion_name,\
                        'ecoregion_nr': ecoregion_nr, 'mean_elevation': ee.Number(elevation.get('be75')),\
                        'eii_sum': eii_plot_sum, 'eii_mean': eii_plot_mean, 'biomass': biomass_plot, 'mean_canopy2020': mean_canopy,\
                        'senW_bii50': bii50,'senW_npp50': npp50, 'senW_lfc50': lfc50, 'senW_bii0': bii0, 'senW_npp0': npp0, 'senW_lfc0': lfc0,\
                        'senA_1km2': eii1, 'senA_2km2': eii2, 'senA_3km2': eii3, 'senA_4km2': eii4}))



## Calculate EII score and its sensitivity for a set of test plots

The values presented in the article are calculated using a set of test plots hereafter called *Seome plots*.

Seome plots are categorised to the following regions: North America (NAmerica), South America (SAmerica), East Asia (EastAsia), South Asia (SAsia), North Asia (NAsia), Australia, Europe and Africa.

There are Seome plots which are located on an island, intersect with plantations or are located above the elevation threshold set in the methods (1000 m above the sea level). These plots were removed in post-processing, the code does not filter them out.

The plots are categorised to Top and Down category in which Top category plots were assessed to contain high quality of forest and expected to have higher EII score.
Some Seome plots have Down category marked as Down or DOWN.

The code may not be able to run all the plots together, it is wise to filter and process the plots region by region.

In [36]:
# filter out middle category
seome_all = ee.FeatureCollection('projects/ee-katrinmschn/assets/seome_plots_public')
# Choose the region which plots to analyse - this is also used for the exported csv file
#region = 'Europe'
#seome = seome_all.filter(ee.Filter.eq('Region', region))

In [39]:
print('Nr of Seome plots to analyse: ', seome_all.size().getInfo())

Nr of Seome plots to analyse:  333


Iterate over all Seome plots and show the output in geopandas dataframe

In [38]:
seome_eii = seome_all.map(eii_sensitivity_seome)
gdf = gm.ee_to_geopandas(seome_eii)
gdf.head()

,geometry,Category,Category_n,Region,biomass,biome,ecoregion,ecoregion_nr,eii_mean,eii_sum,...,senA_1km2,senA_2km2,senA_3km2,senA_4km2,senW_bii0,senW_bii50,senW_lfc0,senW_lfc50,senW_npp0,senW_npp50
0,"POLYGON ((20.86171 -1.23526, 20.86173 -1.23590...",1,Top,Africa,118.155263,Tropical & Subtropical Moist Broadleaf Forests,Central Congolian lowland forests,3,0.937740,4.703410,...,0.919762,1.850476,2.828746,3.758860,4.735044,4.688436,4.548236,4.781840,4.828636,4.641640
1,"POLYGON ((20.35370 -2.04621, 20.35372 -2.04684...",1,Top,Africa,256.418424,Tropical & Subtropical Moist Broadleaf Forests,Central Congolian lowland forests,3,0.945421,4.734519,...,0.945772,1.898373,2.843644,3.792780,4.842047,4.685457,4.602575,4.805192,4.768338,4.722311
2,"POLYGON ((24.66684 -1.64592, 24.66686 -1.64655...",1,Top,Africa,222.077469,Tropical & Subtropical Moist Broadleaf Forests,Central Congolian lowland forests,3,0.969330,4.854250,...,0.972873,1.945883,2.915468,3.884880,4.785279,4.888736,4.777454,4.892649,5.000018,4.781367
3,"POLYGON ((26.13901 -0.08608, 26.13904 -0.08672...",1,Top,Africa,214.501557,Tropical & Subtropical Moist Broadleaf Forests,Northeast Congolian lowland forests,24,0.918922,4.479296,...,0.921767,1.743121,2.629815,3.563611,4.423082,4.507018,4.575428,4.430845,4.438608,4.499255
4,"POLYGON ((16.75669 -0.19595, 16.75671 -0.19658...",1,Top,Africa,NaN,Tropical & Subtropical Moist Broadleaf Forests,Western Congolian swamp forests,29,0.924396,4.636481,...,0.934918,1.862241,2.750256,3.686094,4.697793,4.599183,4.440236,4.727961,4.758129,4.569015


Uncomment to export as file

In [20]:
# uncomment to export the geodataframe to shapefile  - allows to check in QGIS and export as CSV
#gdf.to_file('eii_sensitivity_'+ region + '.shp)

# export the geodataframe to csv
filename =  'exported_eii/eii_score.csv'
#pd.DataFrame(gdf.assign(geometry=gdf["geometry"].apply(lambda p: p.wkt))).to_csv(filename)